In [1]:
# !pip install --trusted-host pypi.org --trusted-host pypi.pyhton.org --trusted-host files.pythonhosted.org -U --upgrade xlrd


In [2]:
import os
import sys 
import pandas as pd
import re
from selenium import webdriver 
import win32com.client
import time
from datetime import date
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import xlsxwriter
import concurrent.futures
from datetime import datetime
import requests

import pdfplumber

import matplotlib.pyplot as plt
import plotly.express as px

import shutil

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
#from bs4 import BeautifulSoup
import PyPDF2
from PyPDF2 import PdfReader

import xlrd

In [3]:
source_directory = r"C:\Users\bd1266\Downloads\Data Download"
backup_directory = r"C:\Users\bd1266\Downloads\Data Download\Back Up"

In [4]:
today = datetime.now().strftime("%Y-%m-%d %H:%M")


In [5]:
def start_driver():

    options = webdriver.ChromeOptions()

    #Select desired download folder.
    download_directory = "C:\\Users\\bd1266\\Downloads\\Data Download"
    prefs = {
                "download.default_directory": download_directory,
                 "download.prompt_for_download": False,
                 "download.directory_upgrade": True,
                 "safebrowsing.enabled": True,
                 "safebrowsing.disable_download_protection": True}
    options.add_experimental_option("prefs", prefs)
    
    # Add headless mode
    #options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    return driver



In [6]:
df_columns = ['Source', 'Date', 'Status', 'Error'] 
hourly_refresh_status_df = pd.DataFrame(columns = df_columns)

# DATOS FINANCIEROS DEL MERCADO LOCAL


## 1.1 Rava

### 1.1 Dolar

In [7]:
def rava_download():
    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://www.rava.com/")
    time.sleep(15)



    

    body_xpath= '/html/body/div[1]/main/div/div/div[2]/div[1]/div[2]/table/tbody'


    list_of_names = ["MEP", "CCL", "MAYORISTA"]
    
    
    
    cell = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)
    # Count the number of child elements
    item_count = len(child_elements)
    print(item_count)
    results =[]

    list_of_names

    df_dolar = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

    # Assuming child_elements is already defined and contains the relevant elements
    for i in child_elements:
        split_values = i.text.split()

        # Ensure there are at least 3 parts before adding to DataFrame
        if len(split_values) >= 3:
            # Check if the first part is in the list_of_names
            if split_values[0] in list_of_names:
                # Check if the second part is not "LEDE" or "SENEBI"
                if split_values[1] not in ["LEDE", "SENEBI"]:
                    # Create a new row as a DataFrame
                    new_row = pd.DataFrame({
                        "Fecha" : today,
                        "Name": [split_values[0]],
                        "Valor": [split_values[1]],
                        "% dia": [split_values[2]]
                    })
                    # Append the new row to the existing DataFrame using concat
                    df_dolar = pd.concat([df_dolar, new_row], ignore_index=True)
                else:
                    print(f"Skipping entry with 'LEDE' or 'SENEBI': {split_values}")
            else:
                print(f"Value not useful: {split_values}")
        else:
            print(f"Not enough parts in split_values: {split_values}")
    driver.quit()    
    return df_dolar

    
# Display the final DataFrame


In [8]:
try:
    df_dolar = rava_download()
    row = {"Source" : "Rava",
               "Date" : today,
               "Status": "Success",
               "Error" : "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except:
    try:
        df_dolar = rava_download()
        row = {"Source" : "Rava",
               "Date" : today,
               "Status": "Success",
               "Error" : "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        row = {"Source" : "Rava",
               "Date" : today,
               "Status": "Error",
               "Error" : e}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open
10
Value not useful: ['OFICIAL', '1.078,75', '-', '-', '-']
Value not useful: ['TARJETA', '1.401,73', '-', '0,4', '2,5']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'LEDE', '1.214,52', '3,3', '3,9', '4,8']
Skipping entry with 'LEDE' or 'SENEBI': ['MEP', 'SENEBI', '1.207,78', '1,6', '3,6', '3,6']
Value not useful: ['OFI-MEP', '12,20', '-', '43,5', '8,9']
Value not useful: ['OFI-CCL', '12,30', '-', '18,3', '-3,9']
Value not useful: ['MEP-CCL', '0,50', '-', '-72,2', '-64,3']


### 1.2 Bonos

In [9]:
def rava_bonos_download():

    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://www.rava.com/cotizaciones/bonos")
    time.sleep(15)





    body_xpath= '/html/body/div[1]/main/div/div/div[2]/table/tbody'


    list_of_names = ["AL30", "AL30D", "AL30C", "GD30", "GD30D", "GD30C" , "BPY26"]



    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)
    # Count the number of child elements
    item_count = len(child_elements)
    print(item_count)
    results =[]

    list_of_names

    df_bonos = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

    # Assuming child_elements is already defined and contains the relevant elements
    for i in child_elements:
        split_values = i.text.split()

        # Ensure there are at least 3 parts before adding to DataFrame
        if len(split_values) >= 3:
            # Check if the first part is in the list_of_names
            if split_values[0] in list_of_names:

                    new_row = pd.DataFrame({
                        "Fecha" : today,
                        "Name": [split_values[0]],
                        "Valor": [split_values[1]],
                        "% dia": [split_values[2]]
                    })
                    # Append the new row to the existing DataFrame using concat
                    df_bonos = pd.concat([df_bonos, new_row], ignore_index=True)

            else:
                pass
        else:
            print(f"Not enough parts in split_values: {split_values}")

    # Display the final DataFrame
    driver.quit()    
    return df_bonos

In [10]:
try:
    df_bonos = rava_bonos_download()
    row = {"Source" : "Rava_Bonos",
            "Date" : today,
            "Status" : "Success",
            "Error" : "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except:
    try:
        df_bonos = rava_download()
        row = {"Source" : "Rava_Bonos",
            "Date" : today,
            "Status" : "Success",
            "Error" : "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        row = {"Source" : "Rava_Bonos",
               "Date" : today,
               "Status" : "Error",
               "Error" : e}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open
62


### 1.3 Lo mejor - Lo peor

In [11]:
def lomejorlopeor():
    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://www.rava.com")
    time.sleep(15)

    body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[8]/div[2]/table/tbody'


    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)

    # Count the number of child elements
    item_count = len(child_elements)
    

    # Initialize an empty DataFrame with specified column names
    df_lomejor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

    # Loop through each child element and extract data
    for i in child_elements:
        split_values = i.text.split()

        new_row = pd.DataFrame({
            "Fecha": [today],
            "Name": [split_values[0]],
            "Valor": [split_values[1]],
            "% dia": [split_values[2]]
        })

        # Append the new row to the existing DataFrame using pd.concat
        df_lomejor = pd.concat([df_lomejor, new_row], ignore_index=True)



    body_xpath = '/html/body/div[1]/main/div/div/div[2]/div[9]/div[2]/table/tbody'


    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)

    # Count the number of child elements
    item_count = len(child_elements)
   

    # Initialize an empty DataFrame with specified column names
    df_lopeor = pd.DataFrame(columns=["Fecha", "Name", "Valor", "% dia"])

    # Loop through each child element and extract data
    for i in child_elements:
        split_values = i.text.split()

        new_row = pd.DataFrame({
            "Fecha": [today],
            "Name": [split_values[0]],
            "Valor": [split_values[1]],
            "% dia": [split_values[2]]
        })

        # Append the new row to the existing DataFrame using pd.concat
        df_lopeor = pd.concat([df_lopeor, new_row], ignore_index=True)

    driver.quit()    
    return df_lopeor, df_lomejor

In [12]:
try:
    df_lopeor, dflomejor  = lomejorlopeor()
    row = {"Source" : "Rava: Lo mejor - Lo peor",
               "Date" : today,
               "Status": "Success",
               "Error" : "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except:
    try:
        df_lopeor, dflomejor  = lomejorlopeor()
        row = {"Source" : "Rava: Lo mejor - Lo peor",
               "Date" : today,
               "Status": "Success",
               "Error" : "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
        
        
    except Exception as e:
        row = {"Source" : "Rava: Lo mejor - Lo peor",
               "Date" : today,
               "Status": "Error",
               "Error" : e}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open


### 1.4 Acciones Merval

In [13]:
def merval_acciones():
    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://www.rava.com/cotizaciones/acciones-argentinas")
    time.sleep(15)

    body_xpath = '/html/body/div[1]/main/div/div/div[2]/table/tbody'

    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)

    # Initialize an empty DataFrame with specified column names
    df_merval_acciones = pd.DataFrame(columns=["Especie", "Lider", "Valor", "% dia"])
    
    # Loop through each child element and extract data
    for i in child_elements:
        split_values = i.text.split()

        new_row = pd.DataFrame({
            "Fecha": [today],
            "Especie": [split_values[0]],
            "Valor": [split_values[1]],
            "% dia": [split_values[2]]
        })

        # Append the new row to the existing DataFrame using pd.concat
        df_merval_acciones = pd.concat([df_merval_acciones, new_row], ignore_index=True)
    
    
    
    driver.quit()    
    return df_merval_acciones




In [14]:
def merval_pesos_usd():
    try:
        driver.quit()
    except:
        print("no driver open")
    
    driver = start_driver()    
    time.sleep(5)
    driver.get("https://www.rava.com")
    time.sleep(15)
    zbody_xpath = '/html/body/div[1]/main/div/div/div[4]/div[2]/div[1]/table/tbody'

    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, zbody_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)

    # Initialize an empty DataFrame with specified column names
    df_merval = pd.DataFrame(columns=["Especie", "Ultimo Valor", "% dia", "% mes", "% año"])

    # Loop through each child element and extract data
    for i in child_elements:
        split_values = i.text.split()
        if "USD" in i.text:
            new_row = pd.DataFrame({
                "Fecha": [today],
                "Especie": [split_values[0] + " " + split_values[1]],
                "Ultimo Valor": [split_values[2]],
                "% dia": [split_values[3]],
                "% mes": [split_values[4]],
                "% año": [split_values[5]],
            })
        else: 
            new_row = pd.DataFrame({
                "Fecha": [today],
                "Especie": [split_values[0]],
                "Ultimo Valor": [split_values[1]],
                "% dia": [split_values[2]],
                "% mes": [split_values[3]],
                "% año": [split_values[4]],
            })

        # Append the new row to the existing DataFrame using pd.concat
        df_merval = pd.concat([df_merval, new_row], ignore_index=True)
    driver.quit()    
    return df_merval

In [15]:

try:
    df_merval_acciones = merval_acciones()
    row = {"Source": "Acciones Merval",
           "Date": today,
           "Status": "Success",
           "Error": "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except Exception as e:
    print(e)
    try:
        df_merval_acciones = df_merval_acciones = merval_acciones()
        row = {"Source": "Acciones Merval",
               "Date": today,
               "Status": "Success",
               "Error": "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        print(e)
        row = {"Source": "Acciones Merval",
               "Date": today,
               "Error": str(e)}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open


In [17]:
try:
    merval_pesos_usd = merval_pesos_usd()
    row = {"Source": "Merval: Pesos - USD",
           "Date": today,
           "Status": "Success",
           "Error":  "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except Exception as e:
    print(e)
    
    try:
        merval_pesos_usd = merval_pesos_usd()
        row = {"Source": "Merval: Pesos - USD",
       "Date": today,
       "Status": "Success",
       "Error": "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        print(e)
        row = {"Source": "Merval: Pesos - USD",
               "Date": today,
               "Status": "Error",
               "Error": str(e)}
               
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open


In [18]:
hourly_refresh_status_df

,Source,Date,Status,Error
0,Rava,2025-02-17 17:40,Success,-
1,Rava_Bonos,2025-02-17 17:40,Success,-
2,Rava: Lo mejor - Lo peor,2025-02-17 17:40,Success,-
3,Acciones Merval,2025-02-17 17:40,Success,-
4,Merval: Pesos - USD,2025-02-17 17:40,Success,-


### MATBA FOREX

In [19]:
def forex():
    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://matbarofex.primary.ventures/")
    time.sleep(15)

    body_xpath = body_xpath = '/html/body/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div/div[1]/div[2]/div[2]'


    cell = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, body_xpath))
    )

    # Get all child elements within the specified cell
    child_elements = cell.find_elements(By.XPATH, './*')  # Get all direct children
    time.sleep(2)

    # Count the number of child elements
    item_count = len(child_elements)
    print(item_count)

    df_forex = pd.DataFrame(columns=[ "Instrumento", "Vol C", "Compra", "Venta", "Vol V","Fecha"])
    for i in range(4):

        split_values = child_elements[i].text.split()

        new_row = pd.DataFrame([{
            "Instrumento": split_values[0],
            "Vol C": split_values[1],
            "Compra": split_values[2],
            "Venta": split_values[3],
            "Vol V": split_values[4],
            "Fecha": today
        }])
        #print(new_row)
        df_forex = pd.concat([df_forex, new_row], ignore_index=True)
    driver.quit()    
    return df_forex

In [20]:
try:
    df_forex = forex()
    row = {"Source": "Forex",
           "Date": today,
           "Status": "Success",
           "Error": "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except Exception as e:
    print(e)
    try:
        df_forex = forex()
        row = {"Source": "Forex",
        "Date": today,
        "Status": "Success",
        "Error": "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        print(e)
        row = {"Source": "Forex",
               "Date": today,
               "Status": "Error",
               "Error": str(e)}
               
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open
39


### Trading Economics

#### Crypto

In [21]:
def crypto():
    try:
        driver.quit()
    except:
        print("no driver open")       

    driver = start_driver()

    time.sleep(5)
    driver.get("https://tradingeconomics.com/crypto")
    time.sleep(15)

    table = driver.find_elements(By.XPATH, '/html/body/form/div[4]/div/div/div[1]/div/table/tbody')[0]
    rows = table.find_elements(By.TAG_NAME, 'tr')
    data = []

    # Iterate through each row and extract data
    for row in rows:
        columns = row.find_elements(By.TAG_NAME, 'td')

        # Extract data from each column based on the specified headers
        crypto = columns[0].text.strip()
        price = columns[1].text.strip()
        day = columns[2].text.strip()
        day_percent = columns[3].text.strip()
        weekly = columns[4].text.strip()
        monthly = columns[5].text.strip()
        ytd = columns[6].text.strip()
        yoy = columns[7].text.strip()
        marketCap = columns[8].text.strip()
        date = today
        data.append([crypto, price, day, day_percent, weekly, monthly, ytd, yoy, marketCap, date])
        # Print the extracted data
        #print(f"crypto: {crypto}, price: {price}, Day: {day}, day_percent: {day_percent}, Weekly: {weekly}, Monthly: {monthly}, YTD: {ytd}, YoY: {yoy}, marketCap: {marketCap},  Date: {date}")

    df_trading_economics_crypto = pd.DataFrame(data, columns=['crypto', 'price', 'Day', 'day_percent', 'Weekly', 'Monthly', 'YTD', 'YoY', 'marketCap', 'Date'])
    driver.quit()
    return df_trading_economics_crypto

In [22]:
try:
    df_crypto = crypto()
    row = {"Source": "Crypto",
           "Date": today,
           "Status": "Success",
           "Error":  "-"}
    row_df = pd.DataFrame([row])
    hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
except Exception as e:
    print(e)
    
    try:
        crypto = crypto()
        row = {"Source": "Crypto",
        "Date": today,
        "Status": "Success",
        "Error":  "-"}
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)
    except Exception as e:
        print(e)
        row = {"Source": "Crypto",
               "Date": today,
               "Status": "Error",
               "Error": str(e)}
               
        row_df = pd.DataFrame([row])
        hourly_refresh_status_df = pd.concat([hourly_refresh_status_df, row_df], ignore_index=True)

no driver open


In [23]:
hourly_refresh_status_df

,Source,Date,Status,Error
0,Rava,2025-02-17 17:40,Success,-
1,Rava_Bonos,2025-02-17 17:40,Success,-
2,Rava: Lo mejor - Lo peor,2025-02-17 17:40,Success,-
3,Acciones Merval,2025-02-17 17:40,Success,-
4,Merval: Pesos - USD,2025-02-17 17:40,Success,-
5,Forex,2025-02-17 17:40,Success,-
6,Crypto,2025-02-17 17:40,Success,-


# -------------------------------------------------------------------------------

In [29]:
with pd.ExcelWriter(f'C:/Users/bd1266/OneDrive - Assurant, Inc/Documents/Projects/Python + Pywin32/Automation F/Results/Hourly Refresh_Started at_{today}.xlsx', engine='xlsxwriter') as writer:
    # Write each DataFrame to a different worksheet
    df_bonos.to_excel(writer, sheet_name='Rava Bonos', index=False)
    df_lopeor.to_excel(writer, sheet_name='Rava Lo Peor', index=False)
    dflomejor.to_excel(writer, sheet_name='Rava Lo Mejor', index=False)
    df_merval_acciones.to_excel(writer, sheet_name='Acciones Merval', index=False)
    merval_pesos_usd.to_excel(writer, sheet_name='Merval - Pesos USD', index=False)
    df_forex.to_excel(writer, sheet_name='Forex', index=False)
    df_crypto.to_excel(writer, sheet_name='Crypto')

In [30]:
df_bonos

,Fecha,Name,Valor,% dia
0,2025-02-17 17:40,AL30,"78.450,00","0,78"
1,2025-02-17 17:40,AL30C,"64,34","-1,09"
2,2025-02-17 17:40,AL30D,"64,83","-1,40"
3,2025-02-17 17:40,BPY26,"115.410,00","1,12"
4,2025-02-17 17:40,GD30,"79.120,00","0,33"
5,2025-02-17 17:40,GD30C,"64,75","-2,26"
6,2025-02-17 17:40,GD30D,"65,40","-2,01"


In [31]:
df_lopeor

,Fecha,Name,Valor,% dia
0,2025-02-17 17:40,DOME,"60,00","-20,1"
1,2025-02-17 17:40,GARO,"269,00","-14,1"
2,2025-02-17 17:40,LOMA,"2.620,00","-8,1"
3,2025-02-17 17:40,BBAR,"7.540,00","-7,7"
4,2025-02-17 17:40,CEPU,"1.500,00","-7,4"


In [37]:
dflomejor

,Fecha,Name,Valor,% dia
0,2025-02-17 17:40,GRIM,"1.200,00","1,7"
1,2025-02-17 17:40,RIGO,"890,00","1,3"
2,2025-02-17 17:40,GAMI,"190,00","-0,6"
3,2025-02-17 17:40,INTR,"380,00","-0,9"
4,2025-02-17 17:40,TXAR,"811,00","-1,0"


In [33]:
df_merval_acciones

,Especie,Lider,Valor,% dia,Fecha
0,Especie,NaN,▼,Último,2025-02-17 17:40
1,ALUA,NaN,"810,00","-1,22",2025-02-17 17:40
2,BBAR,NaN,"7.540,00","-7,71",2025-02-17 17:40
3,BMA,NaN,"10.550,00","-4,31",2025-02-17 17:40
4,BYMA,NaN,"430,00","-4,66",2025-02-17 17:40
5,CEPU,NaN,"1.500,00","-7,41",2025-02-17 17:40
6,COME,NaN,"201,00","-7,05",2025-02-17 17:40
7,CRES,NaN,"1.295,00","-5,82",2025-02-17 17:40
8,CVH,NaN,"6.880,00","-4,58",2025-02-17 17:40
9,EDN,NaN,"2.240,00","-6,08",2025-02-17 17:40


In [34]:
merval_pesos_usd

,Especie,Ultimo Valor,% dia,% mes,% año,Fecha
0,MERVAL,"2.254.188,63","-5,6","-12,1","-11,0",2025-02-17 17:40
1,MERVAL USD,"1.862,03","-6,6","-13,9","-12,8",2025-02-17 17:40


In [35]:
df_forex

,Instrumento,Vol C,Compra,Venta,Vol V,Fecha
0,DLR/FEB25,6.037,"1.067,5","1.068,0",3.541,2025-02-17 17:40
1,DLR/FEB25A,6.000,"1.067,5","1.068,0",8.587,2025-02-17 17:40
2,DLR/MAR25,805,"1.084,0","1.084,5",575,2025-02-17 17:40
3,DLR/MAR25A,1.000,"1.084,0","1.085,0",8.000,2025-02-17 17:40


In [36]:
df_crypto

,crypto,price,Day,day_percent,Weekly,Monthly,YTD,YoY,marketCap,Date
0,Bitcoin,96053,137,-0.14%,-1.52%,-8.08%,2.86%,85.01%,,2025-02-17 17:40
1,Ether,2737.90,74.1,2.78%,2.84%,-17.32%,-17.83%,-3.88%,,2025-02-17 17:40
2,Binance,668.1,5.9,-0.88%,8.07%,-5.99%,-4.29%,90.64%,,2025-02-17 17:40
3,Cardano,0.79958,0.02740,3.55%,12.27%,-27.16%,-5.05%,29.26%,,2025-02-17 17:40
4,Solana,178.8757,9.643,-5.12%,-10.84%,-31.96%,-5.47%,59.22%,,2025-02-17 17:40
5,Ripple,2.66025,0.0696,-2.55%,9.62%,-18.85%,28.10%,375.73%,,2025-02-17 17:40
6,Polkadot,4.88,0.01,-0.11%,0.72%,-30.74%,-26.30%,-37.75%,,2025-02-17 17:40
7,Avalanche,24.94,0.13,0.51%,-2.79%,-36.58%,-29.85%,-38.09%,,2025-02-17 17:40
8,Polygon,0.32,0.01,-1.99%,1.91%,-33.99%,-29.13%,-67.99%,,2025-02-17 17:40
9,Cosmos,4.85,0.09,1.80%,0.71%,-26.75%,-21.50%,-54.14%,,2025-02-17 17:40
